# `Psycopg2` : *sql injection*

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# Purpose & Outcome:

+ GO over some things you DO NOT want to do

+ Show how to remedy `1st order sql injection only`

`I am NOT a cybersecurity person NOR am I a DB Admin` This is to give you ideas and insight of things that can occur.

# `Next Video: MySQL/PSQL casting`

In [6]:
import pandas as pd
import psycopg2 # make connection and interface

# Import the 'config' function from the config_user_dta.py file:
from config_user_dta import config  # call my user credentials

`---------------------------------`

**If you do not want to create init or config files: do something similar to this**

`import psycopg2
conn = psycopg2.connect("dbname=test user=postgres") # Connect to an existing database
cur = conn.cursor() #Open a cursor to perform database operations`

[If you want the old file to create config/Ini files etc](https://github.com/MrFuguDataScience/SendDataFrom-R-Python-to-SQL-PSQL-Mongo/blob/master/HOW%20TO%20SEND%20DATA%20from%20Python%20to%20PSQL.ipynb)

In [7]:
# Establish a connection to the database by creating a cursor object

# Get the config params
params_ = config()

# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)

# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()

conn.autocommit = True  # read documentation understanding when to Use & NOT use (TRUE)

# psycopg2.extras.register_hstore(conn)

# String representation: *`PROBLEMS`*

`SQL and Python Representations` of data are not exactly the same.

+ For example **single qoutes** for a string.
    * In `SQL` is is used as a string literal delimiter and if used inside a string will neeed to be escaped.
    * In `Python` this is not a problem and doesn't matter.
    
While this can seem like a subtle distinction it can have consequences in SQL.

`Assume that you have something you got online from a website or a file that you are entering for some database.` This string of data could have something hidden which would cause great issues for you if it came from a malicious person. Something you should be aware of is a `SQL injection`.

`------------------------`

**<font color=red> For example</font>**: you are looking at a non-profit site that deals with donating money to certain groups or chrarities or people who knows. This is what the site looks like when you click a button and see donations page and then you will see their current campaigns to donate money.

**`https://some_nonprofit.org/donations?group=current_campaigns`**

Now, lets say that they have this webpage may have a query behind the scenes that looks like this:

**`SELECT * FROM donations WHERE group ='current_campaigns' AND current=1;`**

This query will then:

+ select everything from the dontations and find current campaigns that are available (current=1)

What if now we start manipulating this webpage since it is not very secure by chance?

**`https://some_nonprofit.org/donations?group='current_campaigns'`** **<font color=red size=4>--</font>**

The little innocuous piece you see in red will have some horrible effects if this is not a secure website. Let's see what is going on...

**`SELECT * FROM donations WHERE group ='current_campaigns'`** **<font color=red>'--</font>** **`AND current=1;`**


+ The SQL query interprets this as a comment; therefore anything occuring after it will be considered a comment. So the `AND current=1` referencing the current campaigns available are ignored and everything can be made visible 😳. 


<font size=6>☠️</font> **<font color=red>Pirate Party Time</font>**

**Well, if you are a pirate searching for information why stop here?**

`________________________`

**`SELECT * FROM donations WHERE group ='current_campaigns'`** **<font color=red>'--</font>** **`AND current=1;`**


Now you have effectively, gave yourself accesss to things you didn't know existed. Freeing up more data to view or take advantage of. 

`How about another step toward being a true pirate:` don't mind if I do...



**`SELECT * FROM donations WHERE group ='current_campaigns'`** <font color=red size=3>+ OR 1=1--'</font> **`AND current=1;`**

The kicker: this query always evaluates to `TRUE` oops... Now, the data is free to view. 

`---------------------------`


https://www.psycopg.org/docs/usage.html

https://en.wikipedia.org/wiki/SQL_injection

# <font color=red>DO NOT</font> `DO THIS IN PRACTICE: YOU WILL HAVE REAL PROBLEMS!!!!` 

<font size=6>☠️ ☠️ ☠️ ☠️</font>

# `Python with PSQL` (psycopg2): two other examples

`_____________________________________`


# **Ex. ) 1** `String Parameter Interpolation`

`**********************************`



`sql = "INSERT INTO user_info (names) VALUES` <font color=red>('%s')</font> `;"` <font size=6>☠️</font> **Pirate Party Time**

`input_dta=('johnny_wants_toGetHacked')` # and this line could be anything harmful!

`cur.execute(sql` <font color=red>%</font> `input_dta)`

This input statement can get a whole bunch of bad stuff put here because you don't have anyway to prevent malicious attacks. Let's see what we can do here:


`input_dta="'; update user_info set admin = 'true' where names = 'hacker'; select true; --"`

*I mean why not, we are given the ability to put whatever we want. I guess we can update some user and give them admin privileges for later.*


**Why did this happen? I mean that little old** *`'%s'`* seems so insignificant... 

Basically you are letting yourself have a hey day for bad people because you have that single qoute `''` in the sql statement which is interpreted as your query. Because of this you can input not so good things if you understand the underlying structure of these tables 😱.

But, there's something else: the string parameter interpolation at the end:

`cur.execute(sql % input_dta)` our little % is NOT good news either. 


`Looks like somebody is getting fired soon.`


**Now the good news psycopg2** can interpret/automatically convert Python objects to/from SQL literals.

`What?`

`Well you don't need to pass the ('%s') but instead %s` which is a step in the right direction and you would end up with 

`cur.execute(sql,input_dta)` as well. But, we are not out of the woods yet!

# **Ex. 2)** `String Concatenation`

`**************************************`

`query = "SELECT * FROM user_info WHERE group = '"+ input + "'";`

`result_ = cursor.execute(query)`

`cursor.fetchall()`

<font color=red>Oh No, Not Again</font> another issue!

This is because we have the same issue of benig able to re-write the query based on this string concatination. 


# But wait: *I need to tell you something, that good old* <font color=red>where clause</font>  *is another one of our potential problems too*.

# How can we deal with this?

`Well a good thing would be to start with parameterized queries!`

**Hmm, what is that?**

Let's look at some examples to get an idea

# Start thinking about this: 

+ Anytime there is `user input` there is a chance of creating a vulnerability!
    + You will need checks and validation steps to help curb this problem.

# `Today's problem`:

`I have a website that stores Car Dealer information: such as that from the last video. And we have a login page, but we are not entirely sure how to format the webpage and make the site secure because we are newbies and naive.`

*Let's pretend to be a `Pirate named Brown Eye Jerry` like a guy from graduate school I knew. A real <font size=3>💩</font> and he is going to enter our website from the backdoor as all pirates do and exploit my website.*

In [8]:
# CREATE TABLE FOR PSQL: staging_fake_ppl

def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS staging_website_users;
        CREATE UNLOGGED TABLE staging_website_users (
            user_          varchar(30),
            DB_Admin       Boolean
        );""")

In [9]:
# Send the Schema to PSQL

with conn.cursor() as cursor:
    create_staging_table(cursor)

In [15]:
# Insert some fake data:

sql_="""INSERT INTO staging_website_users (user_,DB_Admin) VALUES ('Alf',True),
('Monty',False),('humpty_dumpty',False),('yaya123',False);"""
cur.execute(sql_)

In [16]:
# verify
cur.execute("select * from staging_website_users")
cur.fetchall()

[('Alf', True), ('Monty', False), ('humpty_dumpty', False), ('yaya123', False)]

 `--------------------------`
 
 
 
 <font size=7>☠️ Pirate Party Time</font> 
 
 
 pretend you have this next function to check users trying to enter your system and it is not secure. Leaving our little pirates and trolls to have a fun time

In [19]:
# BAD EXAMPLE. DON'T DO THIS! 
def is_admin(username: str):
    with conn.cursor() as cursor:
        cur.execute("""
            SELECT
                DB_admin
            FROM
                staging_website_users
            WHERE
                user_ = '%s'          
                                    
        """ % username)               #  this line % and '' in line above: NO NO NO
        result = cur.fetchone()

    if result is None:
        # User does not exist
        return False

    admin, = result
    return admin

# https://realpython.com/prevent-python-sql-injection/ (This Function from here)

In [24]:
# Check who is an DB_Admin:
print(is_admin('Alf'))
print(is_admin('yay123'))

True
False
Pirate entered your DB: True


# `Exploit` our DB:

In [31]:

print('Pirate entered your DB:',is_admin("'; select true; --"))

Pirate entered your DB: True


In [26]:
# Print the actual query:
print("select DB_Admin from staging_website_users where user) \
= '%s'" % "'; select true; --")

select DB_Admin from staging_website_users where user) = ''; select true; --'


# What just happened?

+ You would assume the argument you want is a specific user_, but what you are getting instead is: 

`1.)` a query terminated by a semicolon

`2.)` a `select true` statement that is true (always)

`3.)` then a comment `--` which will take care of anything after it like a password because it is now ignored.

**`This would be very bad if someone wanted to delete everything, store something for later etc. Because you could create a way to just enter later if you wanted for do everything now.`**

# Updated  function:

+ This time we do not have single qoutes or the string interpolation

In [29]:
def is_admin_updated(username: str):
    with conn.cursor() as cursor:
        cur.execute("""
            SELECT
                DB_admin
            FROM
                staging_website_users
            WHERE
                user_ = %(username)s
                                    
        """, {
            'username': username
        })
        result = cur.fetchone()
    if result is None:
        # User does not exist
        return False

    admin, = result
    return admin

In [30]:
# updated so pirate don't enter:
print('Pirate entered your DB:',is_admin_updated("'; select true; --"))

Pirate entered your DB: False


# Things to note here:

1.) we created a named parameter which is linked to the `username` 

2.) no more nasty single qoutes `''`


# <font color=red>Don't Do</font>: *bold portions*

+ `"SELECT DB_Admin FROM staging_website_users WHERE user_ =` **'" + username + '**

+ `"SELECT DB_Admin FROM staging_website_users WHERE user_ =`  **'%s'  % username**

+ `"SELECT DB_Admin FROM staging_website_users WHERE user_ =` **{ }'".format(username)**

+ `f"SELECT DB_Admin FROM staging_website_users WHERE user_ =` **'{username}'"**

# Try to Do:

+ `%s", (username, )`

+ `%(username)s", {'username': username}`

# When we are doing queries: `psycopg2` has an advantage over the Python API

we are able to do `Literals & Identifiers` *versus* Python only carrying out `Literals`

**`But what's the difference?`**

* *`Literal`*: strings, numerics, dates

* *`Identifier`*: tables and columns

In [149]:
from psycopg2 import sql #help safely write sql queries, verify you can't put bad stuff!


def find_dta(table_name: str, f_name: str,name:str):
    with conn.cursor() as cursor:
        state_ment = sql.SQL("""
            SELECT
                *
            FROM {table_name}
                WHERE {f_name}={name}
        """).format(
            table_name = sql.Identifier(table_name),
            f_name = sql.Identifier(f_name),
            name=sql.Literal(name)
        )
        cur.execute(state_ment)
        result = cur.fetchall()

    return result


pd.DataFrame(find_dta('staging_fake_ppl','first_name','Anthony'),
             columns=['credit_card','email',
                        'first_name','last_name','phone_number']).head()


# https://www.psycopg.org/docs/sql.html #(please look at)

# http://aklaver.org/wordpress/2018/04/21/building-dynamic-sql-using-psycopg2/ #(examples)

,credit_card,email,first_name,last_name,phone_number
0,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348
1,1280-4584-5250-1133,axthrlcob@kdqvoabtw.nmg,Anthony,Henley,6958349723
2,6921-1083-1797-8702,xzgefkqlh@nhocyb.qwy,Anthony,Hazelman,9315684987
3,5725-6056-1230-2423,kj@tr.jdt,Anthony,Garcia,7244972846
4,8170-8477-1042-3899,rexi@eitlpqv.pce,Anthony,Green,8362748419


In [147]:
# same thing almost:

query = sql.SQL("select * from {table} where {column_ofInterest} = %s").format(
    table=sql.Identifier('staging_fake_ppl'),
    column_ofInterest=sql.Identifier('first_name'))

cur.execute(query,('Anthony',))

pd.DataFrame(cur.fetchall(),columns=['credit_card','email',
                        'first_name','last_name','phone_number']).head()

,credit_card,email,first_name,last_name,phone_number
0,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348
1,1280-4584-5250-1133,axthrlcob@kdqvoabtw.nmg,Anthony,Henley,6958349723
2,6921-1083-1797-8702,xzgefkqlh@nhocyb.qwy,Anthony,Hazelman,9315684987
3,5725-6056-1230-2423,kj@tr.jdt,Anthony,Garcia,7244972846
4,8170-8477-1042-3899,rexi@eitlpqv.pce,Anthony,Green,8362748419


# Other Notable SQL Injections:

go here for an idea: https://portswigger.net/web-security/sql-injection 

and good old: mr Google

# <font color=red>Like</font>, Share &

# <font color=red>SUB</font>scribe

# Citations & Help: 

# <font size=7>◔̯◔</font>

https://www.postgresqltutorial.com/psql-commands/ (sql shortcuts)

`SQL Attacks`:

https://www.veracode.com/security/sql-injection

https://portswigger.net/web-security/sql-injection

https://realpython.com/prevent-python-sql-injection/

https://www.hacksplaining.com/prevention/sql-injection

https://security.berkeley.edu/education-awareness/best-practices-how-tos/system-application-security/how-protect-against-sql

https://tools.cisco.com/security/center/resources/sql_injection

https://bobby-tables.com/python